In [ ]:
from api_key import api_key

In [ ]:
api_key

In [ ]:
import requests

In [ ]:
r = f'http://www.europeana.eu/api/v2/search.json?wskey={api_key}&query=europeana_collectionName="9200300_Ag_EU_TEL_a0600_Newspapers_ONB"'
requests.get(r).content

In [ ]:
r = f'http://www.europeana.eu/api/v2/item/'

In [ ]:
id = '9200300/BibliographicResource_3000123565676'
id = '9200300/BibliographicResource_3000095610170'
r = f'https://api.europeana.eu/record/v2/{id}.json?wskey={api_key}'
d = requests.get(r).json()['object']['proxies'][1]

In [ ]:
!jq -cr '.dcTitle' newspapers.ndjson | tail

In [ ]:
def simplify(obj, key = None):
    if not type(obj) == dict:
        return obj
#    if 'europeanaProxy' in obj:        
#        return {k : simplify(val, key) for k, val in obj.items()}
    r = {}
    for k, v in obj.items():
        if k == "def": 
            r[key] = simplify(obj[k], k)
        else:
            r[key + "-" + k] = simplify(obj[k], k)
    return r
def metadata_for(id):
    r = f'https://api.europeana.eu/record/v2/{id}.json?wskey={api_key}'
    d = requests.get(r).json()['object']['proxies'][1]
    output = {}
    for k, v in d.items():
        item = simplify(v, k)
        if type(item) == dict:
            for k2, v2 in item.items():
                output[k2] = v2
        else:
            output[k] = item
    return output


In [ ]:
from pyarrow import parquet as pq
from pathlib import Path
import json
from pyarrow import json as pajson
import time
seen = set(pajson.read_json("newspapers.ndjson")['id'].to_pylist())

print(len(seen), "seen")
for p in Path("parquet_files/").glob("*.parquet"):
    break
    fin = pq.read_table(p, columns = ['ids'])
    ids = set([*map(lambda x: x.rsplit("/", 1)[0], fin['ids'].to_pylist())])
    with open("newspapers.ndjson", "a") as fout:
        for i, id in enumerate(ids):
            if id in seen:
                continue
            print(f"{i} - {id}", end = "\r")
            try:
                r = metadata_for(id)
            except:
                time.sleep(1)
                continue
                
            r['id'] = id
            fout.write(json.dumps(r) + "\n")


In [4]:
len(f.filelist)

147652

In [ ]:
import zipfile
f = zipfile.ZipFile("9200300.zip")

In [30]:
import json
i = 1000
demo = f.open(f.filelist[i]).read().decode("utf-8")
g = Graph().parse(data=demo, format="xml") #<-took a while to figure this line out!
d = g.serialize(format='json-ld', indent=0)
proxies = [f for f in json.loads(d) if 'http://www.openarchives.org/ore/terms/Proxy' in f['@type']]
proxies[0].keys()

dict_keys(['@id', '@type', 'http://purl.org/dc/elements/1.1/identifier', 'http://purl.org/dc/elements/1.1/language', 'http://purl.org/dc/elements/1.1/source', 'http://purl.org/dc/elements/1.1/subject', 'http://purl.org/dc/elements/1.1/title', 'http://purl.org/dc/elements/1.1/type', 'http://purl.org/dc/terms/extent', 'http://purl.org/dc/terms/isPartOf', 'http://purl.org/dc/terms/issued', 'http://purl.org/dc/terms/spatial', 'http://www.europeana.eu/schemas/edm/europeanaProxy', 'http://www.europeana.eu/schemas/edm/isNextInSequence', 'http://www.europeana.eu/schemas/edm/type', 'http://www.openarchives.org/ore/terms/proxyFor', 'http://www.openarchives.org/ore/terms/proxyIn'])

In [11]:
import zipfile
f = zipfile.ZipFile("9200300.zip")
def parse_row(d):
    proxies = [f for f in json.loads(d) if 'http://www.openarchives.org/ore/terms/Proxy' in f['@type']]
    out = {}
    for k, v in proxies[1].items():
        if "purl.org/dc" in k:
            try:
                out['dc:' + k.split("/")[-1]] = v[0]['@value']
            except KeyError:
                out['dc:' + k.split("/")[-1]] = v[0]['@id']
    return out

"9200300" + f.filelist[i].filename.replace(".xml", "")


'BibliographicResource_3000116299792.xml'

In [3]:

#!pip install rdflib-jsonld

from rdflib import Graph, plugin
from rdflib.serializer import Serializer

import gzip

def write_chunk(start):
    end = start + 25_000
    dest = f"{start}-{end}.json.gz"

    if dest.exists():
        return
    with gzip.open(dest, "wt") as fout:
        for i in range(start, end):
            print(i, end = "\r")
            demo = f.open(f.filelist[i]).read().decode("utf-8")
            g = Graph().parse(data=demo, format="xml") #<-took a while to figure this line out!
            d = g.serialize(format='json-ld', indent=0)
            json.dump(parse_row(d), fout)
            fout.write("\n")

In [ ]:
ls -lSh jsonld.gz

In [ ]:
ls -lSh jsonld.gz

In [ ]:
from pyarrow import json as pajson
pajson.read_json("jsonld.gz")

In [ ]:
import xmltodict


In [ ]:
import rdflib
g = rdflib.graph
#rdflib.parse(f.open("BibliographicResource_3000116299792.xml").read())
g.pa